In [ ]:
pip install prophet

In [ ]:
import pandas as pd
import numpy as np

from prophet import Prophet


congestion = pd.read_csv('/Users/gabrielmedeiros/Desktop/Data_Science_Competitions/Comp1/train.csv')
test_data = pd.read_csv('/Users/gabrielmedeiros/Desktop/Data_Science_Competitions/Comp1/test.csv')

congestion.head()

In [ ]:
test_data.shape

In [ ]:
test_data.tail(10)

In [ ]:
congestion.tail(10)

In [ ]:
# congestion['time'] = pd.to_datetime(congestion['time'])

# congestion['month_name'] = congestion['time'].dt.month_name()

# congestion['month'] = congestion['time'].dt.month

# congestion['day_name'] = congestion['time'].dt.day_name()

# congestion['day'] = congestion['time'].dt.day

# congestion['hour'] = congestion['time'].dt.hour

# congestion['minute'] = congestion['time'].dt.minute

# congestion = congestion.drop(columns = ['time','row_id'])

# congestion = pd.get_dummies(congestion, columns = ['month_name', 'day_name', 'direction'])

# congestion.head()

In [ ]:
df_for_prophet = pd.DataFrame(congestion.time, congestion.congestion).reset_index()
df_for_prophet.columns = ['y', 'ds']
df_for_prophet.tail()

In [ ]:
df_for_prophet.shape[0]

In [ ]:
validation = df_for_prophet.tail(424417)
train = df_for_prophet.head(424418)

In [ ]:
m = Prophet()
m.fit(df_for_prophet)
# future = m.make_future_dataframe(periods=2340, freq = '20min')
# future.tail()



In [ ]:
future = pd.DataFrame(test_data.time)
future.columns = ['ds']
future.head()

In [ ]:
forecast = m.predict(future)

In [ ]:
forecast.head()

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize = (20,15))
plt.plot(forecast.ds, forecast.yhat, color = 'blue')
plt.plot(forecast.ds, forecast.yhat_upper, color = 'red')
plt.plot(forecast.ds, forecast.yhat_lower, color = 'red')
plt.title('Congestion Levels')
plt.xlabel('Date')
plt.ylabel('Congestion')
plt.xticks(rotation = 45)
plt.grid()
plt.show()

In [ ]:
m.plot_components(forecast)

In [ ]:
from prophet.plot import add_changepoints_to_plot
fig = m.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), m, forecast)


## Saving a prophet model and loading using json

In [ ]:

# import json
# from prophet.serialize import model_to_json, model_from_json

# with open('serialized_model.json', 'w') as fout:
#     json.dump(model_to_json(m), fout)  # Save model

# with open('serialized_model.json', 'r') as fin:
#     m = model_from_json(json.load(fin))  # Load model

## Hyperparameter tuning

In [ ]:
import itertools
import numpy as np
import pandas as pd

param_grid = {  
    'changepoint_prior_scale': [0.001, 0.01, 0.1, 0.5],
    'seasonality_prior_scale': [0.01, 0.1, 1.0, 10.0],
}

# Generate all combinations of parameters
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []  # Store the RMSEs for each params here
from tqdm import tqdm
# Use cross validation to evaluate all parameters
for params in tqdm(all_params):
    m = Prophet(**params).fit(validation)  # Fit model with given params
    df_cv = cross_validation(m, cutoffs=cutoffs, horizon='24 hours', parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])

# Find the best parameters
tuning_results = pd.DataFrame(all_params)
tuning_results['rmse'] = rmses
print(tuning_results)

In [ ]:
best_params = all_params[np.argmin(rmses)]
print(best_params)